In [1]:
#Import standard packages
import pandas as pd
import numpy as np
import xarray as xr
import math
import os

from pyaldata import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import io
from scipy import stats
from sklearn.metrics import r2_score
import pickle
from tqdm import tqdm
import csv

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history


#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [2]:
#Get Data in the correct format
#enter the folder where the data is
folder='../preprocessed_data/'

with open(folder+'all_data.pickle','rb') as f:
    #If using python 3
    M1, PMd, M1_PMd,pos_binned,vels_binned=pickle.load(f,encoding='latin1') 


In [3]:
# Function that reassigns different angles into classes from 1 to 8
# going anticlockwise starting from +x direction
def determine_angle(angle):
    if angle == 0:
        return 1
    elif angle == math.pi/4:
        return 2
    elif angle == math.pi/2:
        return 3
    elif angle == 3*math.pi/4:
        return 4
    elif angle == math.pi:
        return 5
    elif angle == -3*math.pi/4:
        return 6
    elif angle == -math.pi/2:
        return 7
    elif angle == -math.pi/4:
        return 8

In [4]:
# Load data
import os
data_dir = '../raw_data/'
fname = os.path.join(data_dir, "Chewie_CO_CS_2016-10-14.mat")

# load TrialData .mat file into a DataFrame
df = mat2dataframe(fname, shift_idx_fields=True)

# Keep only successful trials
df = select_trials(df, "result == 'R'")

# Import the whole data set
# combine time bins into longer ones
td_full = combine_time_bins(df, 3)

# Remove low-firing neurons
td_full = remove_low_firing_neurons(td_full, "M1_spikes",  5)
td_full = remove_low_firing_neurons(td_full, "PMd_spikes", 5)

# Get the signal from idx_go_cue
df.idx_movement_on = df.idx_movement_on.astype(int)
td_full = restrict_to_interval(td_full, start_point_name='idx_go_cue', end_point_name='idx_trial_end')


td_full = smooth_signals(td_full, ["M1_spikes", "PMd_spikes"], std=0.05)

# Get the data in the right format
N_full = td_full.shape[0]

lengths = []
angle = []
for i in range(N_full):
    # get position
    pos = td_full.pos[i]
    lengths.append(pos.shape[0])
    angle.append(determine_angle(td_full.target_direction[i]))

c:\users\jon\documents\biomedical engineering - ic\4. letnik\final year project\python\pyaldata\pyaldata\tools.py:979: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


In [6]:
angle_data = np.zeros(shape=(M1.shape[0],1))
start = 0
for (idx,duration) in enumerate(lengths):
    end = start + duration
    angle_data[start:end] = angle[idx]
    
    
    start = end


print(M1.shape, angle_data.shape)


(30068, 63) (30068, 1)


In [10]:
## Do preprocessing and run the decoder for different neural data sets
#neural_data_combined = [M1, PMd, M1_PMd]
neural_data_combined = [M1]
kinematics = [pos_binned, vels_binned]

# User 
lag=0 #What time bin of spikes should be used relative to the output
#(lag=-1 means use the spikes 1 bin before the output)

# variable which stores R^2 values
R2 = np.empty((3,2))
# index variable
for i in tqdm(range(1)):
    neural_data = neural_data_combined[i]
    
    # Formatting for KF
    # Input
    #The covariate is simply the matrix of firing rates for all neurons over time
    X_kf=neural_data
    # Output
    #The final output covariates include position, velocity, and acceleration
    y_kf=np.concatenate((pos_binned,vels_binned),axis=1)
    
    # Take into account lag (if it applies to the analysis)
    num_examples=X_kf.shape[0]

    #Re-align data to take lag into account
    if lag<0:
        y_kf=y_kf[-lag:,:]
        X_kf=X_kf[0:num_examples+lag,:]
    if lag>0:
        y_kf=y_kf[0:num_examples-lag,:]
        X_kf=X_kf[lag:num_examples,:]
        
    # Splitting data into training/testing/validation sets
    #Set what part of data should be part of the training/testing/validation sets
    training_range=[0, 0.7]
    testing_range=[0.7, 0.85]
    valid_range=[0.85,1]
    
    # Split data for KF
    #Number of examples after taking into account bins removed for lag alignment
    num_examples_kf=X_kf.shape[0]

    #Note that each range has a buffer of 1 bin at the beginning and end
    #This makes it so that the different sets don't include overlapping data
    training_set=np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
    testing_set=np.arange(int(np.round(testing_range[0]*num_examples_kf))+1,int(np.round(testing_range[1]*num_examples_kf))-1)
    valid_set=np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)
    
    angle_train = np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
    angle_valid = np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)
    angle_train = np.expand_dims(angle_train, axis=1)
    angle_valid = np.expand_dims(angle_valid, axis=1)
    
    #Get training data
    X_kf_train=X_kf[training_set,:]
    y_kf_train=y_kf[training_set,:]

    #Get testing data
    X_kf_test=X_kf[testing_set,:]
    y_kf_test=y_kf[testing_set,:]

    #Get validation data
    X_kf_valid=X_kf[valid_set,:]
    y_kf_valid=y_kf[valid_set,:]
    
    #  Preprocessing: Normalization and zero-centering
    #Z-score inputs 
    X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
    X_kf_train_std=np.nanstd(X_kf_train,axis=0)
    X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
    X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
    X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std
    
    X_kf_train = np.hstack((X_kf_train, angle_train))
    X_kf_valid = np.hstack((X_kf_valid, angle_valid))
    
    #Zero-center outputs
    y_kf_train_mean=np.mean(y_kf_train,axis=0)
    y_kf_train=y_kf_train-y_kf_train_mean
    y_kf_test=y_kf_test-y_kf_train_mean
    y_kf_valid=y_kf_valid-y_kf_train_mean
    
    
    # Run the decoders
    # 1. Kalman filter
    #Declare model
    model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter that is set to the default in this example (see ReadMe)

    #Fit model
    model_kf.fit(X_kf_train,y_kf_train)

    #Get predictions
    y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)
    
    R2_pos = r2_score(y_kf_valid[:,0:2],y_valid_predicted_kf[:,0:2], multioutput='variance_weighted')
    R2_vel = r2_score(y_kf_valid[:,2:4],y_valid_predicted_kf[:,2:4], multioutput='variance_weighted')
    
    R2[i,:] = R2_pos, R2_vel

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.78s/it]


In [15]:
# Export the data into a csv file for further analysis
#csv_rowlist = [['','Kalman Filter'], ['R^2 values','Position', 'Velocity'], ['M1', R2[0,0], R2[0,1]], ['PMd',  R2[1,0], R2[1,1]],['M1 & PMd', R2[2,0], R2[2,1]]]

#export_path = '../Results/KF_raw.csv'
#with open(export_path, 'w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerows(csv_rowlist)

In [9]:
print(R2)

[[0.66590544 0.73053529]
 [0.64718672 0.63680953]
 [0.81982633 0.79923576]]


In [11]:
print(R2)

[[0.67963619 0.73138787]
 [0.64718672 0.63680953]
 [0.81982633 0.79923576]]
